In [ ]:
# Célula 1: Configuração e Importações
import pandas as pd
import os
import zipfile
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from shapash import SmartExplainer

# Garante que a pasta docs existe
base_path = os.getcwd()
# Ajuste de caminho dependendo de onde o notebook é executado
if os.path.basename(base_path) == 'notebooks':
    docs_path = os.path.abspath(os.path.join(base_path, '..', 'docs'))
else:
    docs_path = os.path.abspath(os.path.join(base_path, 'docs'))

os.makedirs(docs_path, exist_ok=True)
print(f"Artefatos serão salvos em: {docs_path}")

In [ ]:
# Célula 2: Carga de Dados e Treinamento
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00601/ai4i2020.csv"
df = pd.read_csv(url)

# Tradução
mapa_nomes = {
    'Type': 'Tipo_Equipamento', 'Air temperature [K]': 'Temp_Ar',
    'Process temperature [K]': 'Temp_Processo', 'Rotational speed [rpm]': 'Rotacao_rpm',
    'Torque [Nm]': 'Torque_Nm', 'Tool wear [min]': 'Desgaste_Ferramenta_min',
    'Machine failure': 'Falha'
}
df = df.rename(columns=mapa_nomes).drop(['UDI', 'Product ID', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], axis=1)

X = df.drop('Falha', axis=1)
y = df['Falha']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

encoder = OrdinalEncoder(cols=['Tipo_Equipamento'])
X_train_enc = encoder.fit_transform(X_train)
X_test_enc = encoder.transform(X_test)

# Modelo Robusto (Random Forest)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_enc, y_train)
print("Modelo Random Forest treinado.")

In [ ]:
# Célula 3: Configuração XAI (Shapash)
features_dict = {
    'Temp_Ar': 'Temperatura do Ambiente',
    'Rotacao_rpm': 'Velocidade de Rotação',
    'Torque_Nm': 'Torque do Motor',
    'Desgaste_Ferramenta_min': 'Desgaste da Ferramenta',
    'Tipo_Equipamento': 'Tipo de Maquina',
    'Temp_Processo': 'Temperatura do Processo'
}

xpl = SmartExplainer(
    model=model,
    preprocessing=encoder,
    features_dict=features_dict
)

xpl.compile(x=X_test_enc, y_target=y_test)
print("Explicações compiladas.")

In [ ]:
# Célula 4: Geração Manual dos Artefatos (Bypass de Erros)
print("Gerando arquivos de auditoria...")

# 1. Gráfico Global
file_global = os.path.join(docs_path, "1_Auditoria_Feature_Importance.html")
xpl.plot.features_importance().write_html(file_global)
print(f"-> Salvo: {file_global}")

# 2. Gráfico Local (Primeiro caso de falha encontrado)
casos_falha = y_test[y_test == 1].index
id_caso = casos_falha[0] if len(casos_falha) > 0 else X_test.index[0]

file_local = os.path.join(docs_path, f"2_Auditoria_Caso_Local_ID{id_caso}.html")
xpl.plot.local_plot(index=id_caso).write_html(file_local)
print(f"-> Salvo: {file_local}")

# 3. Métricas
acc = accuracy_score(y_test, model.predict(X_test_enc))
f1 = f1_score(y_test, model.predict(X_test_enc))

texto_metricas = f"""
RELATÓRIO DE AUDITORIA TÉCNICA
Data: 02/02/2026
Modelo: Random Forest

Acurácia: {acc:.4f}
F1-Score: {f1:.4f}
"""
file_metrics = os.path.join(docs_path, "3_Auditoria_Metricas.txt")
with open(file_metrics, "w", encoding="utf-8") as f:
    f.write(texto_metricas)
print(f"-> Salvo: {file_metrics}")

print("\nSUCESSO! Todos os artefatos foram atualizados na pasta 'docs'.")